# Volatility forecasting 

> In this example, we'll forecast the volatility of the S&P 500 using a GARCH model

::: {.callout-warning collapse="true"}

## Prerequesites

This tutorial assumes basic familiarity with StatsForecast. For a minimal example visit the [Quick Start](https://nixtla.github.io/statsforecast/examples/getting_started_short.html) 
:::

## Introduction 

The Generalized Autoregressive Conditional Heteroskedasticity (GARCH) model is used for time series that exhibit non-constant volatility over time. Here volatility refers to the conditional standard deviation. The GARCH(p,q) model is given by 

\begin{equation}
y_t = v_t \sigma_t 
\end{equation}

where $v_t$ is independent and identically distributed with zero mean and unit variance, and $\sigma_t$ evolves according to 

\begin{equation}
\sigma_t^2 = w + \sum_{i=1}^p \alpha_i y^2_{t-i} + \sum_{j=1}^q \beta_j \sigma_{t-j}^2
\end{equation}

The coefficients in the equation above must satisfy the following conditions: 
1. $w>0$, $\alpha_i \geq 0$ for all $i$, and $\beta_j \geq 0$ for all $j$ 
2. $\sum_{k=1}^{max(p,q)} \alpha_k + \beta_k < 1$. Here it is assumed that $\alpha_i=0$ for $i>p$ and $\beta_j=0$ for $j>q$. 

A particular case of the GARCH model is the ARCH model, in which $q=0$. Both models are commonly used in finance to model the volatility of stock prices, exchange rates, interest rates, and other financial instruments. They're also used in risk management to estimate the probability of large variations in the price of financial assets. 

By the end of this tutorial, you'll have a good understanding of how to implement a GARCH or an ARCH model in [StatsForecast](https://nixtla.github.io/statsforecast/) and how they can be used to analyze and predict financial time series data. 

**Outline:**

1. Install libraries 
2. Load and explore the data
3. Train models 
4. Perform time series cross-validation 
5. Evaluate results 
6. Forecast volatility 

::: {.callout-tip}
You can use Colab to run this Notebook interactively <a href="https://colab.research.google.com/github/Nixtla/statsforecast/blob/main/nbs/examples/GARCH_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
::: 

## Install libraries 

We assume that you have StatsForecast already installed. If not, check this guide for instructions on [how to install StatsForecast](https://nixtla.github.io/statsforecast/examples/installation.html) 

Install the necessary packages using `pip install statsforecast`

In [ ]:
%%capture
pip install statsforecast -U

## Load and explore the data



In this tutorial, we'll use the last 5 years of prices from the S&P 500 and several publicly traded companies. The data can be downloaded from Yahoo! Finance using [yfinance](https://github.com/ranaroussi/yfinance). To install it, use `pip install yfinance`. 

In [ ]:
%%capture
pip install yfinance 

We'll also need `pandas` to deal with the dataframes. 

In [ ]:
import yfinance as yf
import pandas as pd 

tickers = ['SPY', 'MSFT', 'AAPL', 'GOOG', 'AMZN', 'TSLA', 'NVDA', 'META', 'NKE', 'NFLX'] 
df = yf.download(tickers, start = '2018-01-01', end = '2022-12-31', interval='1mo') # use monthly prices
df.head()

The data downloaded includes different prices. We'll use the [adjusted closing price](https://help.yahoo.com/kb/SLN28256.html#:~:text=Adjusted%20close%20is%20the%20closing,Security%20Prices%20(CRSP)%20standards.), which is the closing price after accounting for any corporate actions like stock splits or dividend distributions. It is also the price that is used to examine historical returns.

Notice that the dataframe that `yfinance` returns has a [MultiIndex](https://pandas.pydata.org/docs/user_guide/advanced.html), so we need to select both the adjusted price and the tickers. 

In [ ]:
df = df.loc[:, (['Adj Close'], tickers)]
df.columns = df.columns.droplevel() # drop MultiIndex
df = df.reset_index()
df.head()

The input to StatsForecast is a dataframe in [long format](https://www.theanalysisfactor.com/wide-and-long-data/) with three columns: `unique_id`, `ds` and `y`: 

- `unique_id`: (string, int or category) A unique identifier for the series.
- `ds`: (datestamp or int) A datestamp in format YYYY-MM-DD or YYYY-MM-DD HH:MM:SS or an integer indexing time.
- `y`: (numeric) The measurement we wish to forecast.

Hence, we need to reshape the data. We'll do this by creating a new dataframe called `price`. 

In [ ]:
prices = df.melt(id_vars = 'Date')
prices = prices.rename(columns={'Date': 'ds', 'variable': 'unique_id', 'value': 'y'})
prices = prices[['unique_id', 'ds', 'y']]
prices

We can plot this series using the `plot` method of the StatsForecast class. 

In [ ]:
from statsforecast import StatsForecast 

In [ ]:
StatsForecast.plot(prices)

With the prices, we can compute the logarithmic returns of the S&P 500 and the publicly traded companies. This is the variable we're interested in since it's likely to work well with the GARCH framework. The logarithmic return is given by 

$return_t = log \big( \frac{price_t}{price_{t-1}} \big)$

We'll compute the returns on the price dataframe and then we'll create a return dataframe with StatsForecast's format. To do this, we'll need `numpy`. 

In [ ]:
import numpy as np 
prices['rt'] = prices['y'].div(prices.groupby('unique_id')['y'].shift(1))
prices['rt'] = np.log(prices['rt'])

returns = prices[['unique_id', 'ds', 'rt']]
returns = returns.rename(columns={'rt':'y'})
returns 

::: {.callout-warning}

If the order of the data is very small (say $<1e-5$), `scipy.optimize.minimize` might not terminate successfully. In this case, transform the data to a higher order, generate the GARCH or ARCH model, and then transform the data back. 

:::


In [ ]:
StatsForecast.plot(returns)

From this plot, we can see that the returns seem suited for the GARCH framework, since large shocks _tend_ to be followed by other large shocks. This doesn't mean that after every large shock we should expect another one; merely that the probability of a large variance is greater than the probability of a small one. 

## Train models

We first need to import the [GARCH](https://nixtla.github.io/statsforecast/models.html) and the [ARCH](https://nixtla.github.io/statsforecast/models.html) models from `statsforecast.models`, and then we need to fit them by instantiating a new StatsForecast object. Notice that we'll be using different values of $p$ and $q$. In the next section, we'll determine which ones produce the most accurate model using cross-validation. We'll also import the [Naive](https://nixtla.github.io/statsforecast/models.html#naive) model since we'll use it as a baseline. 

In [ ]:
from statsforecast.models import (
    GARCH, 
    ARCH, 
    Naive
)

models = [ARCH(1), 
          ARCH(2), 
          GARCH(1,1),
          GARCH(1,2),
          GARCH(2,2),
          GARCH(2,1),
          Naive()
]

To instantiate a new StatsForecast object, we need the following parameters:

- `df`: The dataframe with the training data.
- `models`: The list of models defined in the previous step.
- `freq`: A string indicating the frequency of the data. Here we'll use **MS**, which correspond to the start of the month. You can see the list of panda's available frequencies [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases). 
- `n_jobs`: An integer that indicates the number of jobs used in parallel processing. Use -1 to select all cores.

In [ ]:
sf = StatsForecast(
    df = returns, 
    models = models, 
    freq = 'MS',
    n_jobs = -1
)

## Perform time series cross-validation

Time series cross-validation is a method for evaluating how a model would have performed in the past. It works by defining a sliding window across the historical data and predicting the period following it. Here we'll use StatsForercast's `cross-validation` method to determine the most accurate model for the S&P 500 and the companies selected. 

This method takes the following arguments: 

- `df`: The dataframe with the training data. 
- `h` (int): represents the h steps into the future that will be forecasted.
- `step_size` (int): step size between each window, meaning how often do you want to run the forecasting process.
- `n_windows` (int): number of windows used for cross-validation, meaning the number of forecasting processes in the past you want to evaluate.

For this particular example, we'll use 4 windows of 3 months, or all the quarters in a year. 

In [ ]:
crossvalidation_df = sf.cross_validation(
    df = returns,
    h = 3,
    step_size = 3,
    n_windows = 4
  )

The `crossvalidation_df` object ia a dataframe with the following columns: 

- `unique_id`: index. 
- `ds`: datestamp or temporal index
- `cutoff`: the last datestamp or temporal index for the `n_windows`.
- `y`: true value
- `"model"`: columns with the model’s name and fitted value.

In [ ]:
crossvalidation_df = crossvalidation_df.reset_index()
crossvalidation_df.rename(columns = {'y' : 'actual'}, inplace = True)
crossvalidation_df.head()

In [ ]:
StatsForecast.plot(returns, crossvalidation_df.drop(['cutoff', 'actual'], axis=1))

A tutorial on cross-validation can be found [here](https://nixtla.github.io/statsforecast/examples/crossvalidation.html). 

## Evaluate results 

To compute the accuracy of the forecasts, we’ll use the mean average error (mae), which is the sum of the absolute errors divided by the number of forecasts. There's an implementation of MAE on [datasetsforecast](https://github.com/Nixtla/datasetsforecast/tree/main/), so we'll install it and then import the mae function. 

In [ ]:
%%capture
pip install datasetsforecast -U 

In [ ]:
from datasetsforecast.losses import mae

The MAE needs to be computed for every window and then it needs to be averaged across all of them. To do this, we'll create the following function. 

In [ ]:
def compute_cv_mae(crossvalidation_df):
    """Compute MAE for all models generated"""
    res = {}
    for mod in models: 
        res[mod] = mae(crossvalidation_df['actual'], crossvalidation_df[str(mod)])
    return pd.Series(res)

In [ ]:
mae_cv = crossvalidation_df.groupby(['unique_id', 'cutoff']).apply(compute_cv_mae)

mae = mae_cv.groupby('unique_id').mean()
mae.style.highlight_min(color = 'lightblue', axis = 1)

Hence, the most accurate model to describe the logarithmic returns of Apple's stock is a GARCH(2,1), and so on. 

## Forecast volatility 

We can now generate a forecast for the next quarter. To do this, we'll use the `forecast` method, which requieres the following arguments:

- `h`: (int) The forecasting horizon.
- `level`: (list[float]) The confidence levels of the prediction intervals
- `fitted` : (bool = False) Returns insample predictions.

In [ ]:
levels = [80, 95] # confidence levels for the prediction intervals 

forecasts = sf.forecast(h=3, level=levels)
forecasts = forecasts.reset_index()
forecasts.head()

With the results of the previous section, we can choose the best model for the S&P 500 and the companies selected. Some of the plots are shown below. Notice that we're using somo additional arguments in the `plot` method: 

- `level`: (list[int]) The confidence levels for the prediction intervals (this was already defined). 
- `unique_ids`: (list[str, int or category]) The ids to plot. 
- `models`: (list(str)). The model to plot. In this case, is the model selected by cross-validation.

In [ ]:
StatsForecast.plot(returns, forecasts, level=levels, unique_ids = ['AAPL'], models = ['GARCH(2,1)'])

In [ ]:
StatsForecast.plot(returns, forecasts, level=levels, unique_ids = ['MSFT'], models = ['ARCH(2)'])

In [ ]:
StatsForecast.plot(returns, forecasts, level=levels, unique_ids = ['NFLX'], models = ['ARCH(1)'])

## References 

- [Engle, R. F. (1982). Autoregressive conditional heteroscedasticity with estimates of the variance of United Kingdom inflation. Econometrica: Journal of the econometric society, 987-1007.](http://www.econ.uiuc.edu/~econ508/Papers/engle82.pdf) 

- [Bollerslev, T. (1986). Generalized autoregressive conditional heteroskedasticity. Journal of econometrics, 31(3), 307-327.](https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=7da8bfa5295375c1141d797e80065a599153c19d)

- [Hamilton, J. D. (1994). Time series analysis. Princeton university press.](https://press.princeton.edu/books/hardcover/9780691042893/time-series-analysis)

- [Tsay, R. S. (2005). Analysis of financial time series. John wiley & sons.](https://www.wiley.com/en-us/Analysis+of+Financial+Time+Series%2C+3rd+Edition-p-9780470414354)